# 라이브러리 로드 및 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import warnings
from ast import literal_eval

warnings.filterwarnings('ignore')
pd.set_option('max_colwidth', 100)

In [1]:
from surprise import SVD

ModuleNotFoundError: No module named 'surprise'

In [2]:
movies = pd.read_csv('./input/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(3)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret...",en,Spectre,A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""name"": ""Danjaq"", ""id"": 10761}, {""name"": ""B24"", ""id"": ...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}, {""iso_3166_1"": ""US"", ""name"": ""United States of ...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}, {""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639...",Released,A Plan No One Escapes,Spectre,6.3,4466


# 데이터 가공하기

In [3]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [4]:
movies_df[['genres', 'keywords']][:2]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is..."


In [5]:
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [6]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


# 장르 컨텐츠 유사도 측정
  

만약 영화 A의 장르가 `[Action, Adventure, Fantasy, Science Fiction]`이고,  
B가 `[Adventure, Fantasy, Action]`으로 되어있다면 어떻게 장르별 유사도를 측정할까?
  
여러가지 방법이 있을 수 있으나, 가장 간단한 방법은 genres를 문자열로 변경한 뒤,
이를 CountVectorizer로 피처 벡터화한 행렬 데이터 값을 코사인 유사도로 비교하는 것입니다.  
genres 컬럼을 기반으로 하는 컨텐츠 기반 필터링은 다음 단계로 구현하겠습니다.  
  
  1. 문자열로 변환된 genres 컬럼을 Count 기반으로 피처 벡터화 변환합니다.
  2. genres 문자열을 피처 벡터ㅗ하 행렬로 변환한 데이터 셋을 코사인 유사도를 통해 비교합니다. 이를 위해 데이터 셋의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성합니다.
  3. 장르 유사도가 높은 영화 중에서 평점이 높은 순으로 영화를 추천합니다.

장르 컬럼을 문자열로 변환한 뒤, 사이킷런의 CountVectorizer를 이용해 피처 벡터 행렬로 만들겠습니다.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환.

movies_df['genres_literal'] = movies_df['genres'].apply(lambda x: (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


CountVectorizer로 변환해 4803개의 레코드와 276개의 개별 단어 피처로 구성된 피처 벡터 행렬을 만듬.  
이렇게 생성된 행렬에 사이킷런의 cosine_similarity()를 이용해 코사인 유사도를 계산하겠습니다.  

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_mat.shape)
print(genre_sim[:2])

(4803, 276)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


cosine_similarities() 호출로 생성된 genre_sim 객체는 movies_df 의 genre_literal 컬럼을 피처 벡터화한 행럴(genre_mat) 데이터의 행별로  
유사도 정보를 가지고 있으며, 결국은 movies_df DataFrame의 행별 장르 유사도 값을 가지고 있는 것입니다. movies_df를 장르 기준으로  
컨텐츠 기반 필터링ㄹ을 수행하려면 movies_df의 개별 레코드에 대해서 가장 장르 유사도가 높은 순으로 다른 레코드를 추출해야 하는데,  
이를 위해 앞에서 생성한 genre_sim 객체를 이용합니다.
  
  genre_sim 객체의 기준 행별로 비교 대상이 되는 행의 유사도 값이 높은 순으로 정렬된 행렬의 위치 인덱스 값을 추출하면 됩니다. 값이 높은 순으로 정렬도니 비교 대상 행의 유사도 값이 아니라
  비교 대상 행의 위치 인덱스임에 주의합시다. numpydml argsort() 함수를 이용하여 argsort()[:, ::1] 을 이용하면 유사도가 높은 순으로 정리된 genre_sim 객체의 배교 행 위치 인덱스 값을 간편하게 얻을 수 있습니다.
  genre_sim.argsort()[:, ::1] 을 사용해 높은 순으로 정렬된 비교 행 위치 인덱스 값을 가져오고 그 중에 0번 레코드의 비교 행 위치 인덱스 값만 샘플로 추출해 보겠습니다.

In [9]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

In [10]:
genre_sim_sorted_ind

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401],
       ...,
       [4800, 3809, 1895, ..., 2229, 2230,    0],
       [4802, 1594, 1596, ..., 3204, 3205,    0],
       [4802, 4710, 4521, ..., 3140, 3141,    0]], dtype=int64)

첫번째 값을 보면, 0번 레코드의 경우, 자신인 0번이 가장 가깝고, 그다음 3494번째가 가깝고, 그다음 813이 가깝다는 뜻.  
두번째 값을 보면, 이상하게도 자신의 값이 아닌 262가 가장 가깝다고 나옴.  

## 장르 컨텐츠 필터링을 이용한 영화추천

In [13]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):

    # 인자로 입력된 movies_df 데이터프레임에서 'title' 컬럼이 입력된 title_name 값인 데이터프레임을 추출
    title_movie = df[df['title'] == title_name]


    # title_named을 가진 데이터프레임의 index 객체를 ndarray로 변환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    # 추출된 top_n index 출력. top_n index 는 2차원 데이터임.
    # 데이터프레임에서 index로 사용하기 위해서 1차원 array 로 변경.

    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [14]:
# 위 함수를 이용해 영화 '대부'와 장르별로 유사한 영화 10개를 추천해보겠습니다.

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


대부 2 추천으로 봐서 잘된것도 같지만, Light Sleeper 같이 이상한 영화도 추천함. 좀 더 개선이 필요함을 알 수 있음.  
게다가 Mi America의 경우 평점이 0점임을 알 수 있음.  
이번에는 일단 좀 더 많은 후보군을 선정한 뒤 영화의 평점에 따라 필터링해서 최종 추천하는 방식으로 변경하겠습니다.  
영화의 평점 정보인 'vote_average' 값을 이용하겠습니다.  
그런데 1명, 2명의 소수 관객이 특정 영화에 만점이나 매우 높은 평점을 부여해 왜곡된 데이터가 발생할 수 있습니다. 이를 확인하기 위해 
오름차순으로 정렬해 확인해 보겠습니다.

In [15]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


앞서 말했듯이 왜곡된 데이터가 존재함을 확인할 수 있습니다.  
이를 회피하기 위해 새로운 평가방식이 필요.  
평가횟수에 대한 가중치가 부여된 평점(Weighted Rating) 방식을 사용하겠습니다.  
  
가중 평점 = $\frac{v}{(v+m)}$ `*` R + $\frac{m}{(v+m)}$ `*` C

각 변수의 의미
- v: 개별 영화에 평점을 투표한 횟수, movies_df의 'vote_count'값
- m: 평점을 부여하기 위한최수 투표 횟수, 가중치를 직접 조절하는 역할을 하는데, m 값을 높이면 평점 투표횟수가 많은 영화에 더 많은 가중 평점을 부여, m 값은 전체 투표 횟수에서 상위 60% 에 해당하는 횟수를 기준으로 정하겠습니다.
- R: 개별 영화에 대한 평균 평점, 'vote_average'
- C: 전체 영화에 대한 평균 평점 'movies_df['vote_average'].mean()



In [16]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print(C, m)

6.092171559442016 370.1999999999998


기존 평점을 새로운 가중 평점으로 변경하는 함수를 생성하고 이를 이용해, 새로운 평점 정보인 'vote_weighted' 값을 만들겠습니다.
함수명은 weighted_vote_average()입니다. 이 함수는 데이터 프레임의 레코드를 인자로 받아 이 레코드를 vote_count와 vote_average(), 미리 추출된 m과 C 값을  
적용해 레코드별 가중 평점을 반환합니다. 

In [ ]:
percentile = 0.6
m = movies['vote_count'].quantile(perventile)
C = movies['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']

    return ((v/(v+m)) * R) + ((m/(m+v)) * C)

movies['weighted_vote'] = movies.apply(weighted_vote_average, axis = 1)

In [ ]:
movies_df.sort_values('weighted_vote', ascending=False)[:10]

새롭게 정의된 평점 기준에 따라서 영화를 추천해 보겠습니다. 장르 유사성이 높은 영화를 top_n의 2배수 만큼 후보군으로 선정한 뒤,  
weighted_vote 컬럼 값이 높은 순으로 top_n만큼 추출하는 방식으로 find_sim_movie() 함수를 변경합니다.

In [ ]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values

    # top_n의 2배에 해당하는 장르 유사서이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(tip_n*2)]
    similar_indexes = similar_indexes.reshape(-1)

    # 기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]

    # top_n의 2배에 해당하는 후보군에서 weighted_vote 가 높은 순으로 top_n만큼 추출
    
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_indexes.head(10)

In [ ]:
이전보다 훨씬 나은 영화가 추천되었으나,  
장르만으로 영화가 전달하는 많은 요소와 분위기, 그리고 개인이 좋아하는 성향을 반영하기가 부족할 수 있다.  
이에 좀 더 다양한 컨텐츠 기반으로 확장할 수 있으나, 여기까지 하고,  

이후, 아이템 기반 최근접 이웃 협업 필터링을 구현해 보겠다.